In [1]:
using Revise, DynamicHMC, PosteriorDB, Random, StanLogDensityProblems, JSON, BridgeStan, DataFrames, LinearAlgebra, StatsBase, Distributions
using Plots, ColorSchemes, LaTeXStrings, Statistics, CSV, AdvancedHMC, DynamicObjects
Plots.theme(:default)
const BS = BridgeStan;

In [88]:
pdb = PosteriorDB.database()
model = PosteriorDB.model(pdb, "eight_schools_noncentered")
post = PosteriorDB.posterior(pdb, "eight_schools-eight_schools_noncentered")
impl = PosteriorDB.implementation(model, "stan")
path = PosteriorDB.path(impl)
data = PosteriorDB.dataset(post)
s = PosteriorDB.load(data, String);
bs_model = BS.StanModel(stan_file=path, data=s);
ref = PosteriorDB.reference_posterior(post)
df=DataFrame(PosteriorDB.load(ref));

In [178]:
using TransformVariables
using TransformedLogDensities, LogDensityProblemsAD
t = as((μ = asℝ, σ = asℝ₊, τ = asℝ₊, θs = as(Array, 8)))
dimension(t)
P = TransformedLogDensity(t, lp)
∇P = ADgradient(:ForwardDiff, lp);

p = StanProblem(bs_model);
D = 10;
initial_θ = rand(D);
n_samples, n_adapts = 8000, 2000;
metric = DiagEuclideanMetric(D);
hamiltonian = Hamiltonian(metric, lp);
initial_ϵ = find_good_stepsize(hamiltonian, initial_θ);
integrator = Leapfrog(initial_ϵ);
proposal = NUTS{MultinomialTS, GeneralisedNoUTurn}(integrator);
adaptor = StanHMCAdaptor(MassMatrixAdaptor(metric), StepSizeAdaptor(0.8, integrator));
samples, stats = sample(hamiltonian, proposal, initial_θ, n_samples, adaptor, n_adapts; progress=true);

Sampling   0%|                               |  ETA: 0:12:02
  iterations:                                   2
  ratio_divergent_transitions:                  0.0
  ratio_divergent_transitions_during_adaption:  0.5
  n_steps:                                      1
  is_accept:                                    true
  acceptance_rate:                              0.0
  log_density:                                  -9.008526251177598
  hamiltonian_energy:                           12.708457112487306
  hamiltonian_energy_error:                     0.0
  max_hamiltonian_energy_error:                 4.275140685799279e14
  tree_depth:                                   0
  numerical_error:                              true
  step_size:                                    4.6305929745559276
  nom_step_size:                                4.6305929745559276
  is_adapt:                                     true



















Sampling 100%|███████████████████████████████| Time: 0:00:00
  iterations:                                   8000
  ratio_divergent_transitions:                  0.0
  ratio_divergent_transitions_during_adaption:  0.0
  n_steps:                                      7
  is_accept:                                    true
  acceptance_rate:                              0.9996631866288145
  log_density:                                  -7.883221140637642
  hamiltonian_energy:                           12.261434336830948
  hamiltonian_energy_error:                     -0.10168988877392415
  max_hamiltonian_energy_error:                 -0.36220975447912984
  tree_depth:                                   3
  numerical_error:                              false
  step_size:                                    0.5139431750108344
  nom_step_size:                                0.5139431750108344
  is_adapt:                                     false
  mass_matrix:                 

┌ Info: Finished 8000 sampling steps for 1 chains in 0.62266691 (s)
│   h = Hamiltonian(metric=DiagEuclideanMetric([0.9342838452400835, 0.8845 ...]), kinetic=GaussianKinetic())
│   κ = HMCKernel{AdvancedHMC.FullMomentumRefreshment, Trajectory{MultinomialTS, Leapfrog{Float64}, GeneralisedNoUTurn{Float64}}}(AdvancedHMC.FullMomentumRefreshment(), Trajectory{MultinomialTS}(integrator=Leapfrog(ϵ=0.514), tc=GeneralisedNoUTurn{Float64}(10, 1000.0)))
│   EBFMI_est = 0.9935941009273499
│   average_acceptance_rate = 0.807194288588557
└ @ AdvancedHMC /home/meenaljhajharia/.julia/packages/AdvancedHMC/P0nla/src/sampler.jl:246


In [181]:
constrained_draws = hcat([vcat(col...) for col in eachcol(df)]...)
unc_sample_array = vcat([
    param_unconstrain(bs_model, collect(row))' for row in eachrow(constrained_draws)
]...);

In [187]:
lol = hcat(samples...)'

8000×10 adjoint(::Matrix{Float64}) with eltype Float64:
 -1.70031     -0.906714    0.734783  …  -1.05038    1.53333    0.297404
 -1.70031     -0.906714    0.734783     -1.05038    1.53333    0.297404
 -0.460517     0.755663    0.710915      0.311726   1.77159    1.43333
  0.703879     0.0977349  -0.238458     -0.587269   2.41556    0.151544
 -0.533563     0.206773    0.213283      0.5945     1.5524     1.51525
 -0.278016     1.6936     -0.200963  …   0.987746   2.75352    1.30771
 -0.269133     2.10975    -0.404464     -0.16193    2.66466    0.0730169
  0.920135    -1.58881     0.861389      0.15569    5.61373    1.51151
 -0.310143     1.9703     -0.776694      0.0727206  3.18435    0.991354
  0.0399313   -1.34357    -0.219118      0.632224   0.543323   0.933771
  ⋮                                  ⋱                        
  0.902257     0.824063   -0.4926       -0.986713   5.28713   -1.06266
 -0.906564     1.52395     1.16922      -0.126758   4.84196    1.55081
  0.808308    -0.03678

In [190]:
unc_sample_array

10000×10 Matrix{Float64}:
 10.6803    9.71771     7.77507   …  13.7361     9.33885   0.584418
  6.45384   4.41031     7.0133        3.74315    3.69197   0.767165
 -2.24163   0.761705   -4.74619      -2.25603   -1.01337   0.830766
  2.46002   4.44563     4.31378       3.4816     2.66779   0.535441
  5.62489   3.70945     6.11008       7.07705    4.04478   0.236075
  7.50337  10.2793      4.94622   …  11.9474     7.19249   1.24666
  7.816    -0.746158    6.79006       8.223      6.13427   1.51619
  5.0193    8.18897     9.54053       7.11512    7.30456   1.29114
 10.0454   11.8259      8.17946       3.71629    6.62392   1.35851
  5.18654   7.70174     2.6935        4.82504    6.47553   0.764239
  ⋮                               ⋱                       
  2.78882   0.0901486   1.3044        2.16824    3.87727   1.13078
  5.85363  11.1192     -3.62347       0.246698   6.69739   1.61771
  7.38423   8.7937      4.80441       6.40357    6.40207   1.3757
  2.12388   1.89619     1.47158       1

In [189]:
mean(unc_sample_array)

4.387735631575855